In [63]:
from build_Case1 import build_Case1
from build_Case2 import build_Case2
import numpy as np
from scipy.interpolate import interp1d
import pickle
import pandas as pd
import uuid
import time
from pathlib import Path
import io
import os
import json

rootPath = '/nobackup/jakravit/data/'
runPath = rootPath + 'HE60/run/'
batch_name = 'case2V1'

jpath = '/home4/jakravit/MODTRAN6/TEMP_20cases.json'
with open(jpath, 'r') as f:
    tp = json.load(f)
    
with open('TEST01.json', 'r') as f:
    tp2 = json.load(f)

# phytoplankton SIOP spectral library
phyPath = rootPath+'EAP_phytoplankton_dataset/PFTs_v0817.p'
with open(phyPath, 'rb') as fp:
    phy_library = pickle.load(fp)

# NAP spectral libraries
napPath = rootPath+'EAP_NAP_dataset/NAP_v0817_or.p'
with open(napPath, 'rb') as fp:
    datanap = pickle.load(fp)  

# detpath = rootPath+'EAP_NAP_dataset/det_V1.p'
# with open(detpath, 'rb') as fp:
#     datadet = pickle.load(fp) 

# Benthic library
benthic_lib = pd.read_csv(rootPath+'benthic_spec_library_FT_V0826.csv')

# adjacency library
adj_lib = pd.read_csv(rootPath+'adjacency_spectra_V2.csv')

# aeronet library
aero_lib = pd.read_csv(rootPath+'aeronet_invdata_match.csv')

# modtran json input template
with open('TEST01.json', 'r') as f:
    modtp = json.load(f)

iops, col, row = build_Case1(phy_library, datanap, benthic_lib, adj_lib, aero_lib)

batch_name = 'case1_v1'
uid = 'sdf897sl'

print ('DONE!')

DONE!


In [64]:
for i,k in enumerate(tp['MODTRAN']):
    
    # randomly tweek atm parameters for each case
    aero = aero_lib.sample(n=1)
    # aero = aero[0]
    atm_prof = np.random.choice(['ATM_TROPICAL','ATM_MIDLAT_SUMMER','ATM_MIDLAT_WINTER',
                                  'ATM_SUBARC_SUMMER','ATM_SUBARC_WINTER', 'ATM_US_STANDARD_1976'])
    aero_prof = np.random.choice(['AER_RURAL', 'AER_RURAL_DENSE', 'AER_MARITIME_NAVY', 'AER_MARITIME',
                                   'AER_URBAN','AER_TROPOSPHERIC', 'AER_DESERT'])
    OZA = np.random.choice(range(10,60,5))
    OAA = np.random.choice(range(60,120,5))
    SAA = iops['Geo']['SAA']
    SZA = iops['Geo']['SZA']
    SRA = SAA - OAA
    if SRA < 0:
        SRA = SRA + 360
    GNDALT = np.random.choice(np.linspace(0,4,40)),
    GNDALT = round(GNDALT[0],2)
    Wv = np.random.choice(np.linspace(0, 5, 100))
    Wv = round(Wv,2)
    vis = (aero['AOD_Coincident_Input[440nm]'] + aero['AOD_Coincident_Input[675nm]']) / 2 
    vis = round(vis.values[0],2)
    
    # name
    k['MODTRANINPUT']['NAME'] = f'a{i}'
    # description
    k['MODTRANINPUT']['DESCRIPTION'] = f'{batch_name}_{uid}'
    # case
    k['MODTRANINPUT']['CASE'] = i
    # RT options
    k['MODTRANINPUT']['RTOPTIONS'] = {
        'MODTRN': 'RT_MODTRAN',
        'LYMOLC': False,
        'T_BEST': False,
        'IEMSCT': 'RT_SOLAR_AND_THERMAL',
        'IMULT': 'RT_DISORT',
        'DISALB': True,
        'NSTR': 8,
        'SOLCON': 0.0}
    # atmosphere
    k['MODTRANINPUT']['ATMOSPHERE'] = {
        'MODEL': atm_prof,
        'CO2MX': 390.0,
        'H2OSTR': Wv,
        'H2OUNIT': 'a',
        'O3STR': 0.29,
        'O3UNIT': 'a',}
    # aerosols
    k['MODTRANINPUT']['AEROSOLS'] = {
        'CDASTM': 'B',
        'ASTMX': aero['Angstrom_Exponent_440-870nm_from_Coincident_Input_AOD'].values[0],
        'IHAZE': aero_prof,
        'ARUSS': 'USS',
        'VIS': -vis,
        'IPH': 0,
        'SSALB': {
            'NSSALB': 4,
            'AWAVLN': [0.400, 0.675, 0.870, 1.02],
            'ASSALB': [aero['Single_Scattering_Albedo[440nm]'].values[0], aero['Single_Scattering_Albedo[675nm]'].values[0],
                       aero['Single_Scattering_Albedo[870nm]'].values[0], aero['Single_Scattering_Albedo[1020nm]'].values[0]]},
        'IREGSPC': [{'IREG': 1,
            'NARSPC': 7,
            'AWCCON': 0.0,
            'AERNAM': '',
            'VARSPC': [440.0, 675.0, 870.0, 1020.0, 0.0, 0.0, 0.0],
            'EXTC': [aero['AOD_Extinction-Total[440nm]'].values[0], aero['AOD_Extinction-Total[675nm]'].values[0], 
                   aero['AOD_Extinction-Total[870nm]'].values[0], aero['AOD_Extinction-Total[1020nm]'].values[0], 0.0, 0.0, 0.0],
            'ABSC': [aero['Absorption_AOD[440nm]'].values[0], aero['Absorption_AOD[675nm]'].values[0], 
                   aero['Absorption_AOD[870nm]'].values[0], aero['Absorption_AOD[1020nm]'].values[0], 0.0, 0.0, 0.0],
            'ASYM': [aero['Asymmetry_Factor-Total[440nm]'].values[0], aero['Asymmetry_Factor-Total[675nm]'].values[0], 
                   aero['Asymmetry_Factor-Total[870nm]'].values[0], aero['Asymmetry_Factor-Total[1020nm]'].values[0], 0.0, 0.0, 0.0]}]}
    # geometries
    k['MODTRANINPUT']['GEOMETRY'] = {
        'ITYPE': 3,
        'H1ALT': 18.0,
        'H2ALT': GNDALT + .0001,
        'OBSZEN': 180 - OZA,
        'IPARM': 12,
        'PARM1': SRA,
        'PARM2': SZA}
    # surface
    k['MODTRANINPUT']['SURFACE'] = {
        'SURFTYPE': 'REFL_LAMBER_MODEL',
        'GNDALT': GNDALT,
        'NSURF': 2,
        'SURFP': {'CSALB': 'LAMB_OCEAN_WATER'},
        'SURFA': {'CSALB': 'LAMB_OCEAN_WATER'}}
    # spectral
    k['MODTRANINPUT']['SPECTRAL'] = {
        'V1': 390.0,
        'V2': 962.0,
        'DV': 0.05,
        'FWHM': 0.1,
        'YFLAG': 'R',
        'XFLAG': 'N',
        'FLAGS': 'NTAA  T',
        'MLFLX': 1}
    # file options
    k['MODTRANINPUT']['FILEOPTIONS'] = {
        'JSONOPT': 'WRT_OUTPUT', 'CSVPRNT': f'{batch_name}_{uid}.csv', 'JSONPRNT': f'{batch_name}_{uid}.json'}

In [65]:
tp['MODTRAN'][0]['MODTRANINPUT']

{'NAME': 'a0',
 'DESCRIPTION': 'case1_v1_sdf897sl',
 'CASE': 0,
 'RTOPTIONS': {'MODTRN': 'RT_MODTRAN',
  'LYMOLC': False,
  'T_BEST': False,
  'IEMSCT': 'RT_SOLAR_AND_THERMAL',
  'IMULT': 'RT_DISORT',
  'DISALB': True,
  'NSTR': 8,
  'SOLCON': 0.0},
 'ATMOSPHERE': {'MODEL': 'ATM_US_STANDARD_1976',
  'CO2MX': 390.0,
  'H2OSTR': 0.96,
  'H2OUNIT': 'a',
  'O3STR': 0.29,
  'O3UNIT': 'a'},
 'AEROSOLS': {'CDASTM': 'B',
  'ASTMX': 1.335401,
  'IHAZE': 'AER_DESERT',
  'ARUSS': 'USS',
  'VIS': -0.36,
  'IPH': 0,
  'SSALB': {'NSSALB': 4,
   'AWAVLN': [0.4, 0.675, 0.87, 1.02],
   'ASSALB': [0.9212, 0.9328, 0.9072, 0.8983]},
  'IREGSPC': [{'IREG': 1,
    'NARSPC': 7,
    'AWCCON': 0.0,
    'AERNAM': '',
    'VARSPC': [440.0, 675.0, 870.0, 1020.0, 0.0, 0.0, 0.0],
    'EXTC': [0.453, 0.2597, 0.1819, 0.1384, 0.0, 0.0, 0.0],
    'ABSC': [0.035688, 0.017456, 0.016869, 0.014073, 0.0, 0.0, 0.0],
    'ASYM': [0.746776, 0.666411, 0.620855, 0.599788, 0.0, 0.0, 0.0]}]},
 'GEOMETRY': {'ITYPE': 3,
  'H1ALT': 1